In [1]:
!git clone https://github.com/lanzhang128/multi_agent_autoformalization.git
%cd multi_agent_autoformalization
!mkdir test_results

Cloning into 'multi_agent_autoformalization'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 118 (delta 56), reused 6 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (118/118), 5.50 MiB | 4.63 MiB/s, done.
Resolving deltas: 100% (56/56), done.
/content/multi_agent_autoformalization


In [2]:
!pip install rank_bm25

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!tar -xzf /content/drive/MyDrive/Isabelle2024_linux.tar.gz -C /content/
print('Extraction finished.')

Extraction finished.


In [4]:
import os
os.environ['ISABELLE_DIRPATH'] = os.path.abspath('/content/Isabelle2024')

import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llm import OpenAILLM

gpt_4o = OpenAILLM(
    api_key='api_key',
    model='gpt-4o')

In [6]:
from agent import AutoformalizationAgent

formal_language = 'Isabelle/HOL'
agent_auto = AutoformalizationAgent(
    llm=gpt_4o,
    formal_language=formal_language)

In [7]:
informal = (
    "Definition of Softmax Function: Formally, the standard (unit) softmax function "
    "\\(\\sigma\\colon \\mathbb{R}^K \\to (0, 1)^K\\), where \\(K \\ge 1\\), takes a vector "
    "\\(\\mathbf{z} = (z_1, \\dotsc, z_K) \\in \\mathbb{R}^K\\) and computes each component of vector "
    "\\(\\sigma(\\mathbf{z}) \\in (0, 1)^K\\) with \\(\\sigma(\\mathbf{z})_i = \\frac{e^{z_i}}{\\sum_{j=1}^K e^{z_j}}\\,.\\)"
)

In [8]:
zero_formalization, _ = agent_auto(
    informal_statement=informal)
print(zero_formalization)

definition softmax :: "real list \<Rightarrow> real list" where
  "softmax z = 
    let exp_z = map exp z;
        sum_exp_z = sum_list exp_z
    in map (\<lambda>zi. zi / sum_exp_z) exp_z"



In [9]:
from agent import HardCritiqueAgent
agent_hard = HardCritiqueAgent(
    formal_language=formal_language,
    file_dir="test_results")

logging file is at: test_results/isabelle.log
Isabelle server started with info: server "test" = 127.0.0.1:42317 (password "e2a0a32d-c83f-4284-9a1c-c4514b9f367d")
 in 2.65s.
Isabelle server session 8e698df7-c156-46d1-b0d6-ecefc9d6f610 started in 14.40s.


In [10]:
correctness, error_details = agent_hard(
    formalization=zero_formalization,
    file_prefix="zero")
print(correctness)
print(error_details)

Isabelle server use_theory ended in 2.46s.
False
Identified error on line: 4
Error message: Undefined type name: "real"\<^here>
Failed to parse type




In [11]:
from agent import ImportRetrievalAgent
agent_imports = ImportRetrievalAgent(
    formal_language=formal_language,
    retriever="bm25")
import_formalization = agent_imports(
    formalization=zero_formalization, top_n=1)
print(import_formalization)

imports
"HOL.Complex"
Main
begin
definition softmax :: "real list \<Rightarrow> real list" where
  "softmax z = 
    let exp_z = map exp z;
        sum_exp_z = sum_list exp_z
    in map (\<lambda>zi. zi / sum_exp_z) exp_z"

end


In [12]:
correctness, error_details = agent_hard(
    formalization=import_formalization,
    file_prefix="imp")
print(correctness)
print(error_details)

Isabelle server use_theory ended in 2.28s.
False
Identified error on line: 6
Error message: Inner syntax error\<^here>
Failed to parse prop




In [13]:
from agent import FormalRefinementAgent
agent_formal = FormalRefinementAgent(
    llm=gpt_4o,
    formal_language=formal_language,
    category="none")
detailed_refinement, response = agent_formal(
    informal_statement=informal,
    refinement_mode="detailed",
    formalization_file="test_results/imp.thy",
    correctness=correctness,
    error_details=error_details)
print(detailed_refinement)

theory Softmax
imports
  "HOL.Real"
begin

definition softmax :: "real list \<Rightarrow> real list" where
  "softmax z = 
    let exp_z = map exp z;
        sum_exp_z = sum_list exp_z
    in map (\<lambda>zi. zi / sum_exp_z) exp_z"

end



In [14]:
aspect_description='whether the formalized code involves all mathematical concepts in the natural language statement.'

In [15]:
from agent import SoftCritiqueAgent

agent_soft = SoftCritiqueAgent(
    llm=gpt_4o,
    name='soft critique',
    formal_language=formal_language,
    aspect_description=aspect_description)

In [16]:
aspect_evaluation, _ = agent_soft(
    informal_statement=informal,
    formalization=import_formalization)
print(aspect_evaluation)

Judgement: False
Explanation: The formalization provided in Isabelle/HOL does not fully capture all the mathematical concepts in the natural language statement. The natural language statement defines the softmax function \(\sigma\colon \mathbb{R}^K \to (0, 1)^K\), which means it maps a vector from \(\mathbb{R}^K\) to a vector in \((0, 1)^K\). The formal code, however, defines the softmax function to take a "real list" and return a "real list". While this is a reasonable representation of vectors in Isabelle/HOL, the code does not explicitly ensure that the output is within the interval \((0, 1)\) as required by the definition. Additionally, the code does not explicitly handle the condition \(K \ge 1\), which is an important part of the definition. Therefore, the formalization does not fully involve all the mathematical concepts described in the natural language statement.



In [17]:
from agent import InformalRefinementAgent

agent_informal = InformalRefinementAgent(
    llm=gpt_4o,
    formal_language=formal_language)
informal_refinement, _ = agent_informal(
    informal_statement=informal,
    formalization=import_formalization,
    aspect_description=aspect_description,
    aspect_evaluation=aspect_evaluation)
print(informal_refinement)

imports
  "HOL.Real"
begin

definition softmax :: "real list \<Rightarrow> real list" where
  "softmax z = 
    (if length z \<ge> 1 then
      let exp_z = map exp z;
          sum_exp_z = sum_list exp_z
      in map (\<lambda>zi. zi / sum_exp_z) exp_z
    else [])"

lemma softmax_output_in_01: 
  assumes "length z \<ge> 1"
  shows "\<forall>zi \<in> set (softmax z). 0 < zi \<and> zi < 1"
proof -
  let ?exp_z = "map exp z"
  let ?sum_exp_z = "sum_list ?exp_z"
  have "?sum_exp_z > 0" using assms by (simp add: sum_list_pos)
  then show ?thesis
    using assms by (auto simp: softmax_def Let_def)
qed

end



In [18]:
agent_hard.theorem_prover.terminate()

Isabelle server 127.0.0.1:42317 shutdown.
